# Data Cleaning

## Introduction

This notebook goes through a necessary step of any data science project - data cleaning. Data cleaning is a time consuming and unenjoyable task, yet it's a very important one. Keep in mind, "garbage in, garbage out". Feeding dirty data into a model will give us results that are meaningless.

Specifically, we'll be walking through:

1. **Getting the data - **in this case, we'll be scraping data from a website
2. **Cleaning the data - **we will walk through popular text pre-processing techniques
3. **Organizing the data - **we will organize the cleaned data into a way that is easy to input into other algorithms

The output of this notebook will be clean, organized data in two standard text formats:

1. **Corpus** - a collection of text
2. **Document-Term Matrix** - word counts in matrix format

## Problem Statement

As a reminder, my goal is to look at hundred greatest of all time poems and note their similarities and differences. Specifically, I'd like to know if Lewis Carroll's poem is different than other poets.

## Getting The Data

Luckily, there https://discoverpoetry.com/poems/100-most-famous-poems/

In [2]:
# Web scraping, pickle imports
import requests
from bs4 import BeautifulSoup
import pickle

# Scrapes transcript data from discoverypoetry.com
def url_to_urls():
    '''Returns individual links from https://discoverpoetry.com/poems/100-most-famous-poems/'''
    baseurl = "https://discoverpoetry.com/poems/100-most-famous-poems/"
    page = requests.get(baseurl).text
    soup = BeautifulSoup(page, "lxml")
    authors = []
    links = []
    title = []
    for tag in soup.find_all('a',{"class":"poem-listing-link"}):
        if tag.get('href') == "#":
            continue
        else:
            links.append(tag.get('href'))
            authors.append(tag.find(class_="poem-listing-author").text)
            title.append(tag.find(class_="poem-listing-title").text)
    return authors, links, title

# Scrapes transcript data from scrapsfromtheloft.com
def url_to_transcript(url):
    '''Returns transcript data specifically from all the links scraped from https://discoverpoetry.com/poems/100-most-famous-poems/'''
    page = requests.get(url).text
    soup = BeautifulSoup(page, "lxml")
    text = [p.text for p in soup.find(class_="poem-text").find_all('p')]
    print(url)
    return text


In [3]:
#     result = url_to_urls()
#     authors = result[0]
#     urls = result[1]
#     titles = result[2]
    
#     transcripts = [url_to_transcript(u) for u in urls]

In [4]:
#     # Pickle files for later use

# #   Make a new directory to hold the text files  
# #   !mkdir transcripts    
    
#     for i, c in enumerate(authors):
#         authors[i] = re.sub('[\.\,\—]', '', c[3:])
#         titles[i] = re.sub('[\.\,\—]', '', titles[i])
#         with open("transcripts/" + titles[i] + ".txt", "wb") as file:
#             pickle.dump(transcripts[i], file)

#     with open("transcripts/" + "authors" + ".txt", "wb") as file:
#         pickle.dump(authors, file)

#     with open("transcripts/" + "titles" + ".txt", "wb") as file:
#         pickle.dump(titles, file)

In [5]:
 # Load pickled files
authors = []
with open("transcripts/" + "authors" + ".txt", "rb") as file:
    authors = pickle.load(file)

titles = []
with open("transcripts/" + "titles" + ".txt", "rb") as file:
    titles = pickle.load(file)

print(authors)
print(titles)

data = {}
for i, c in enumerate(titles):
    with open("transcripts/" + c + ".txt", "rb") as file:
        data[c] = pickle.load(file)

print(data['Ozymandias'])


['Edgar Allan Poe', 'Percy Bysshe Shelley', 'Robert Frost', 'Edgar Allan Poe', 'William Ernest Henley', 'Robert Frost', 'Walt Whitman', 'Robert Frost', 'John Donne', 'Emily Dickinson', 'T S Eliot', 'William Wordsworth', 'Samuel Taylor Coleridge', 'Henry Wadsworth Longfellow', 'Rudyard Kipling', 'Samuel Taylor Coleridge', 'John McCrae', 'Emily Dickinson', 'John Keats', 'Oliver Wendell Holmes', 'Elizabeth Barrett Browning', 'William Shakespeare', 'George Gordon Lord Byron', 'Matthew Arnold', 'Alfred Lord Tennyson', 'William Cullen Bryant', 'John Keats', 'Robert Frost', 'Alfred Lord Tennyson', 'Robert Burns', 'Emma Lazarus', 'William Wordsworth', 'Robert Frost', 'John Keats', 'Paul Laurence Dunbar', 'Edgar Allan Poe', 'William Blake', 'Emily Dickinson', 'Percy Bysshe Shelley', 'Christopher Marlowe', 'Robert Service', 'Robert Frost', 'Anne Bradstreet', 'Alfred Lord Tennyson', 'Emily Dickinson', 'Walt Whitman', 'John Keats', 'Henry Wadsworth Longfellow', 'John Donne', 'Alfred Lord Tennyson'

In [ ]:
# Double check to make sure data has been loaded properly
data.keys()

In [ ]:
# More checks
count = 0
for x in enumerate(data.items()): 

    # enumerate function returns a tuple in the form 
    # (index, (key, value)) it is a nested tuple 
    # for accessing the value we do indexing x[1][1] 
        count += 1 
print(count) 

## Cleaning The Data

When dealing with numerical data, data cleaning often involves removing null values and duplicate data, dealing with outliers, etc. With text data, there are some common data cleaning techniques, which are also known as text pre-processing techniques.

With text data, this cleaning process can go on forever. There's always an exception to every cleaning step. So, we're going to follow the MVP (minimum viable product) approach - start simple and iterate. Here are a bunch of things you can do to clean your data. We're going to execute just the common cleaning steps here and the rest can be done at a later point to improve our results.

**Common data cleaning steps on all text:**
* Make text all lower case
* Remove punctuation
* Remove numerical values
* Remove common non-sensical text (/n)
* Tokenize text
* Remove stop words

**More data cleaning steps after tokenization:**
* Stemming / lemmatization
* Parts of speech tagging
* Create bi-grams or tri-grams
* Deal with typos
* And more...

In [ ]:
# Let's take a look at our data again
next(iter(data.keys()))

In [ ]:
# Notice that our dictionary is currently in key: poet, poem: list of text format
next(iter(data.values()))

In [6]:
# We are going to change this to key: comedian, value: string format
def combine_text(list_of_text):
    '''Takes a list of text and combines them into one large chunk of text.'''
    combined_text = ' '.join(list_of_text)
    return combined_text

In [7]:
# Combine it!
data_combined = {key: [combine_text(value)] for (key, value) in data.items()}

In [8]:
# We can either keep it in dictionary format or put it into a pandas dataframe
import pandas as pd
pd.set_option('max_colwidth',150)
data_df = pd.DataFrame.from_dict(data_combined, orient='index')
data_df.columns = ['transcript']
# data_df = data_df.sort_index()
data_df

,transcript
The Raven,"\nOnce upon a midnight dreary,\nWhile I pondered, weak and weary,\r\n Over many a quaint and curious\nVolume of forgotten l..."
Ozymandias,"\r\n I met a traveller from an antique land\r\n Who said: ""Two vast and trunkless legs of stone\r\n ..."
The Road Not Taken,"\r\n Two roads diverged in a yellow wood,\r\n And sorry I could not travel both\r\n ..."
Annabel Lee,"\r\n It was many and many a year ago,\nIn a kingdom by the sea,\r\n That a maiden there lived whom y..."
Invictus,"\r\n Out of the night that covers me,\nBlack as the pit from pole to pole,\r\n I thank whatever gods..."
...,...
Evening Star,"\r\n ’Twas noontide of summer,\nAnd mid-time of night;\r\n And stars, in their orbits,\nShone pale, ..."
A Late Walk,"\r\n When I go up through the mowing field,\nThe headless aftermath,\r\n Smooth-laid like thatch wit..."
Ae Fond Kiss,"\r\n Ae fond kiss, and then we sever;\r\n Ae fareweel, and then forever!\r\n ..."
My Shadow,"\r\n I have a little shadow that goes in and out with me,\r\n And what can be the use of him is more..."


In [9]:
# Let's take a look at the transcript for Ali Wong
data_df.transcript.loc['Ozymandias']

'\r\n                        I met a traveller from an antique land\r\n                        Who said: "Two vast and trunkless legs of stone\r\n                        Stand in the desert. Near them on the sand,\r\n                        Half sunk, a shattered visage lies, whose frown\r\n                        And wrinkled lip and sneer of cold command\r\n                        Tell that its sculptor well those passions read\r\n                        Which yet survive, stamped on these lifeless things,\r\n                        The hand that mock\'d them and the heart that fed;\r\n                        And on the pedestal these words appear:\r\n                        \'My name is Ozymandias, king of kings:\r\n                        Look on my works, ye Mighty, and despair!\'\r\n                        Nothing beside remains. Round the decay\r\n                        Of that colossal wreck, boundless and bare,\r\n                        The lone and level sands stretch far a

In [10]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', ' ', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [11]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_df.transcript.apply(round1))
data_clean

,transcript
The Raven,\nonce upon a midnight dreary \nwhile i pondered weak and weary \r\n over many a quaint and curious\nvolume of forgotten l...
Ozymandias,\r\n i met a traveller from an antique land\r\n who said two vast and trunkless legs of stone\r\n ...
The Road Not Taken,\r\n two roads diverged in a yellow wood \r\n and sorry i could not travel both\r\n ...
Annabel Lee,\r\n it was many and many a year ago \nin a kingdom by the sea \r\n that a maiden there lived whom y...
Invictus,\r\n out of the night that covers me \nblack as the pit from pole to pole \r\n i thank whatever gods...
...,...
Evening Star,\r\n ’twas noontide of summer \nand mid time of night \r\n and stars in their orbits \nshone pale ...
A Late Walk,\r\n when i go up through the mowing field \nthe headless aftermath \r\n smooth laid like thatch wit...
Ae Fond Kiss,\r\n ae fond kiss and then we sever \r\n ae fareweel and then forever \r\n ...
My Shadow,\r\n i have a little shadow that goes in and out with me \r\n and what can be the use of him is more...


In [12]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('\r', ' ', text)
    text = re.sub('—', ' ', text)    
    return text

round2 = lambda x: clean_text_round2(x)

In [13]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_clean.transcript.apply(round2))
data_clean

,transcript
The Raven,once upon a midnight dreary while i pondered weak and weary over many a quaint and curious volume of forgotten lore ...
Ozymandias,i met a traveller from an antique land who said two vast and trunkless legs of stone ...
The Road Not Taken,two roads diverged in a yellow wood and sorry i could not travel both ...
Annabel Lee,it was many and many a year ago in a kingdom by the sea that a maiden there lived whom you ma...
Invictus,out of the night that covers me black as the pit from pole to pole i thank whatever gods may ...
...,...
Evening Star,twas noontide of summer and mid time of night and stars in their orbits shone pale thro th...
A Late Walk,when i go up through the mowing field the headless aftermath smooth laid like thatch with the...
Ae Fond Kiss,ae fond kiss and then we sever ae fareweel and then forever deep i...
My Shadow,i have a little shadow that goes in and out with me and what can be the use of him is more tha...


**NOTE:** This data cleaning aka text pre-processing step could go on for a while, but we are going to stop for now. After going through some analysis techniques, if you see that the results don't make sense or could be improved, you can come back and make more edits such as:
* Mark 'cheering' and 'cheer' as the same word (stemming / lemmatization)
* Combine 'thank you' into one term (bi-grams)
* And a lot more...

## Organizing The Data

I mentioned earlier that the output of this notebook will be clean, organized data in two standard text formats:
1. **Corpus - **a collection of text
2. **Document-Term Matrix - **word counts in matrix format

### Corpus

We already created a corpus in an earlier step. The definition of a corpus is a collection of texts, and they are all put together neatly in a pandas dataframe here.

In [14]:
# Let's take a look at our dataframe
data_df

,transcript
The Raven,"\nOnce upon a midnight dreary,\nWhile I pondered, weak and weary,\r\n Over many a quaint and curious\nVolume of forgotten l..."
Ozymandias,"\r\n I met a traveller from an antique land\r\n Who said: ""Two vast and trunkless legs of stone\r\n ..."
The Road Not Taken,"\r\n Two roads diverged in a yellow wood,\r\n And sorry I could not travel both\r\n ..."
Annabel Lee,"\r\n It was many and many a year ago,\nIn a kingdom by the sea,\r\n That a maiden there lived whom y..."
Invictus,"\r\n Out of the night that covers me,\nBlack as the pit from pole to pole,\r\n I thank whatever gods..."
...,...
Evening Star,"\r\n ’Twas noontide of summer,\nAnd mid-time of night;\r\n And stars, in their orbits,\nShone pale, ..."
A Late Walk,"\r\n When I go up through the mowing field,\nThe headless aftermath,\r\n Smooth-laid like thatch wit..."
Ae Fond Kiss,"\r\n Ae fond kiss, and then we sever;\r\n Ae fareweel, and then forever!\r\n ..."
My Shadow,"\r\n I have a little shadow that goes in and out with me,\r\n And what can be the use of him is more..."


In [15]:
# Let's add the authors full names as well
data_df['author'] = authors
data_df

,transcript,author
The Raven,"\nOnce upon a midnight dreary,\nWhile I pondered, weak and weary,\r\n Over many a quaint and curious\nVolume of forgotten l...",Edgar Allan Poe
Ozymandias,"\r\n I met a traveller from an antique land\r\n Who said: ""Two vast and trunkless legs of stone\r\n ...",Percy Bysshe Shelley
The Road Not Taken,"\r\n Two roads diverged in a yellow wood,\r\n And sorry I could not travel both\r\n ...",Robert Frost
Annabel Lee,"\r\n It was many and many a year ago,\nIn a kingdom by the sea,\r\n That a maiden there lived whom y...",Edgar Allan Poe
Invictus,"\r\n Out of the night that covers me,\nBlack as the pit from pole to pole,\r\n I thank whatever gods...",William Ernest Henley
...,...,...
Evening Star,"\r\n ’Twas noontide of summer,\nAnd mid-time of night;\r\n And stars, in their orbits,\nShone pale, ...",Edgar Allan Poe
A Late Walk,"\r\n When I go up through the mowing field,\nThe headless aftermath,\r\n Smooth-laid like thatch wit...",Robert Frost
Ae Fond Kiss,"\r\n Ae fond kiss, and then we sever;\r\n Ae fareweel, and then forever!\r\n ...",Robert Burns
My Shadow,"\r\n I have a little shadow that goes in and out with me,\r\n And what can be the use of him is more...",Robert Louis Stevenson


In [16]:
# Let's pickle it for later use
data_df.to_pickle("corpus.pkl")

### Document-Term Matrix

For many of the techniques we'll be using in future notebooks, the text must be tokenized, meaning broken down into smaller pieces. The most common tokenization technique is to break down text into words. We can do this using scikit-learn's CountVectorizer, where every row will represent a different document and every column will represent a different word.

In addition, with CountVectorizer, we can remove stop words. Stop words are common words that add no additional meaning to text such as 'a', 'the', etc.

In [17]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean.transcript)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm

,abash,abated,abbot,abed,abide,abides,ablution,abode,abodes,abora,...,youth,youthful,youths,youve,ypsilantis,zenith,zigzag,zone,zones,ægean
The Raven,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ozymandias,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
The Road Not Taken,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Annabel Lee,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Invictus,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Evening Star,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A Late Walk,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ae Fond Kiss,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
My Shadow,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# Let's pickle it for later use
data_dtm.to_pickle("dtm.pkl")

In [19]:
# Let's also pickle the cleaned data (before we put it in document-term matrix format) and the CountVectorizer object
data_clean.to_pickle('data_clean.pkl')
pickle.dump(cv, open("cv.pkl", "wb"))

## Additional Exercises

1. Can you add an additional regular expression to the clean_text_round2 function to further clean the text?
2. Play around with CountVectorizer's parameters. What is ngram_range? What is min_df and max_df?